# Collect all settled locations

In [519]:
import geopandas as gpd
import pandas as pd
import geopy
import osmnx as ox
import numpy as np
import rasterio as rio
iq_locations = gpd.read_file('data/settled_locations/irq_pplp_ocha_20140722.shp').sample(500)
location_csv = gpd.GeoDataFrame()
location_csv['name'] = iq_locations['PNameEn']
location_csv['region'] = iq_locations['A1NameEn']
location_csv['country'] = 'Iraq'
location_csv['lat'] = iq_locations['Latitude']
location_csv['lon'] = iq_locations['Longitude']
location_csv['location_type'] = 'town'
location_csv['conflict_date'] = 0
location_csv['pop/cap'] = 0
location_csv.head()

,name,region,country,lat,lon,location_type,conflict_date,pop/cap
10705,Door al-Zera'a,Thi-Qar,Iraq,31.093889,46.244722,town,0,0
10750,Hoor Saiedah M/6,Thi-Qar,Iraq,31.109444,45.909444,town,0,0
14314,Grd Diwan,Erbil,Iraq,35.660000,43.510000,town,0,0
6936,Pasha City,Sulaymaniyah,Iraq,35.577550,45.343450,town,0,0
17446,Al-Nasriya,Kirkuk,Iraq,35.430333,44.114806,town,0,0


# Get population numbers

In [509]:
pop87 = pd.read_csv('data/undata_1987_census/UNdata_Export_20180505_224541956.csv')
pop87 = pop87[:-2] # drop nans

In [528]:
pop87

,Country or Area,Year,Area,Sex,City,City type,Record Type,Reliability,Source Year,Value,Value Footnotes
0,Iraq,1987,Total,Both Sexes,Adhamiyah,City proper,Census - de facto - complete tabulation,"Final figure, complete",1987.0,464151.0,NaN
1,Iraq,1987,Total,Both Sexes,Amara,City proper,Census - de facto - complete tabulation,"Final figure, complete",1987.0,208797.0,NaN
2,Iraq,1987,Total,Both Sexes,BAGHDAD,City proper,Census - de facto - complete tabulation,"Final figure, complete",1987.0,3841268.0,1.0
3,Iraq,1987,Total,Both Sexes,Basra,City proper,Census - de facto - complete tabulation,"Final figure, complete",1987.0,406296.0,NaN
4,Iraq,1987,Total,Both Sexes,Diwaniya,City proper,Census - de facto - complete tabulation,"Final figure, complete",1987.0,196519.0,NaN
5,Iraq,1987,Total,Both Sexes,Erbil,City proper,Census - de facto - complete tabulation,"Final figure, complete",1987.0,485968.0,NaN
6,Iraq,1987,Total,Both Sexes,Hilla,City proper,Census - de facto - complete tabulation,"Final figure, complete",1987.0,268834.0,NaN
7,Iraq,1987,Total,Both Sexes,Kadhimain,City proper,Census - de facto - complete tabulation,"Final figure, complete",1987.0,521444.0,NaN
8,Iraq,1987,Total,Both Sexes,Karradah Sharqiyah,City proper,Census - de facto - complete tabulation,"Final figure, complete",1987.0,235554.0,NaN
9,Iraq,1987,Total,Both Sexes,Kerbala,City proper,Census - de facto - complete tabulation,"Final figure, complete",1987.0,296705.0,NaN


In [511]:
'''iq_pop_bydist = gpd.read_file('data/Iraq_Districts/iraq_districts.shp')
iq_pop_bydist['num_settlements'] = 0
iq_pop_bydist['pop87'] = 0
iq_pop_bydist['pop_2015'] = 0
iq_pop_bydist['pred_pop'] = 0

replace_dict = {}
# this had to be done by hand to check spellings and varied Eng-Arabic transliteration
replace_dict['Adhamiya'] = pop87.Value[pop87.City== 'Adhamiyah']
replace_dict['Amara'] = pop87.Value[pop87.City== 'Amara']
# districts not given unique counts in 87: 
replace_dict['Tarmia'] = pop87.Value[pop87.City== 'Baghdad'] / 
replace_dict['Khadamiya'] = pop87.Value[pop87.City== 'Kadhimain']
replace_dict[''] = pop87.Value[pop87.City== 'Karradah Sharqiyah']'''

"iq_pop_bydist = gpd.read_file('data/Iraq_Districts/iraq_districts.shp')\niq_pop_bydist['num_settlements'] = 0\niq_pop_bydist['pop87'] = 0\niq_pop_bydist['pop_2015'] = 0\niq_pop_bydist['pred_pop'] = 0\n\nreplace_dict = {}\n# this had to be done by hand to check spellings and varied Eng-Arabic transliteration\nreplace_dict['Adhamiya'] = pop87.Value[pop87.City== 'Adhamiyah']\nreplace_dict['Amara'] = pop87.Value[pop87.City== 'Amara']\n# districts not given unique counts in 87: \nreplace_dict['Tarmia'] = pop87.Value[pop87.City== 'Baghdad'] / \nreplace_dict['Khadamiya'] = pop87.Value[pop87.City== 'Kadhimain']\nreplace_dict[''] = pop87.Value[pop87.City== 'Karradah Sharqiyah']"

In [512]:
x = np.array([[1, 2, 3, 4], [4,4,4,4]])
x.shape

(2, 4)

# Above was taking too long, so just going to use worldpop estimates,  crs matches for locations and worldpop raster

In [520]:
# 2015 projections from WorldPop, adjusted to UN estimates
pops_raster = rio.open('data/IRQ_ppp_v2b_2015_UNadj/IRQ_ppp_v2b_2015_UNadj.tif')


In [514]:
'''#http://www.perrygeo.com/python-affine-transforms.html
pop_pred = []
pops_dat = pops.read().squeeze()
translator = pops.affines
with open('pops_pred.txt', 'w') as f:
    for y in range(pops_dat.shape[0]):
        for x in range(pops_dat.shape[1]):
            long, lat = translator * (x,y) # remember to swap b/c this is x,y
            f.write(",".join([str(long), str(lat), str(pops_dat[y,x])]))
# takes too long, try another way'''

'#http://www.perrygeo.com/python-affine-transforms.html\npop_pred = []\npops_dat = pops.read().squeeze()\ntranslator = pops.affines\nwith open(\'pops_pred.txt\', \'w\') as f:\n    for y in range(pops_dat.shape[0]):\n        for x in range(pops_dat.shape[1]):\n            long, lat = translator * (x,y) # remember to swap b/c this is x,y\n            f.write(",".join([str(long), str(lat), str(pops_dat[y,x])]))\n# takes too long, try another way'

In [521]:
translator = pops_raster.affine
pops_dat = pops_raster.read().squeeze()
iq_locs_pop = location_csv.copy()
pops = []
import numpy as np
for i in iq_locs_pop.itertuples():
    x = i.lon
    y = i.lat
    col, row = ~ translator * (x, y)
    col = int(round(col,0))
    row = int(round(row,0))
    pops.append(int(pops_dat[row, col] * 10))
    
location_csv['pop_density'] = np.arange(0, len(location_csv))
location_csv.to_csv('data/iq_locs_pop.csv')

In [522]:
location_csv.head(20)

,name,region,country,lat,lon,location_type,conflict_date,pop/cap,pop_density
10705,Door al-Zera'a,Thi-Qar,Iraq,31.093889,46.244722,town,0,0,0
10750,Hoor Saiedah M/6,Thi-Qar,Iraq,31.109444,45.909444,town,0,0,1
14314,Grd Diwan,Erbil,Iraq,35.660000,43.510000,town,0,0,2
6936,Pasha City,Sulaymaniyah,Iraq,35.577550,45.343450,town,0,0,3
17446,Al-Nasriya,Kirkuk,Iraq,35.430333,44.114806,town,0,0,4
11066,As Sayyid Turfah,Thi-Qar,Iraq,31.810000,45.970000,town,0,0,5
15920,Kas Abu Riwiyah al-Gharbi,Kerbala,Iraq,32.434167,44.210000,town,0,0,6
21705,Al-Bokshy,Wassit,Iraq,32.268056,46.305833,town,0,0,7
19012,Hatim As Sayhudi,Missan,Iraq,31.710000,47.230000,town,0,0,8
12151,Shandrukh,Diyala,Iraq,33.330000,45.350000,town,0,0,9


# now getting the routes between all locations
start with just doing great circle distance, then maybe move to routing values

In [523]:
dist_iq_locations = iq_locations.copy()

In [ ]:
'''dist_df = pd.DataFrame()
for i in dist_iq_locations.itertuples():
    dist_df[i.PNameEn] = dist_iq_locations.geometry.distance(i.geometry)
dist_df.index = dist_df.columns
dist_df.to_csv('distance_matrix.csv')'''

In [524]:
trips = set()
count = 0
with open("flee-release/routes.csv", "w") as f:
    #f.write(",".join(["name1", "name2", "distance\n"]))
    for i in dist_iq_locations.itertuples():
        dists = dist_iq_locations.geometry.distance(i.geometry) * 1000
        dists = [int(round(i,0)) for i in dists]
        tups = zip(dist_iq_locations.PNameEn, dists)
        for destination, time in tups:
            key = (i.PNameEn, destination)
            if key[0] == key[1]:
                continue
            if key not in trips:
                trips.add(key)
                trips.add((key[1], key[0]))
                f.write(",".join([key[0], key[1], str(time)]) + "\n")
                if key[0] != key[1]:
                    f.write(",".join([key[1], key[0], str(time)]) + "\n")

# conflict locations for location_csv

In [ ]:
'''conflict_loc = pd.read_csv('https://data.humdata.org/hxlproxy/data.csv?url=https%3A%2F%2Fapi.acleddata.com%2Facled%2Fread.csv%3Flimit%3D0%26iso%3D368&name=ACLEDHXL&tagger-match-all=on&tagger-02-header=iso&tagger-02-tag=%23country%2Bcode&tagger-03-header=event_id_cnty&tagger-03-tag=%23event%2Bcode&tagger-05-header=event_date&tagger-05-tag=%23date%2Boccurred+&tagger-08-header=event_type&tagger-08-tag=%23event%2Btype&tagger-09-header=actor1&tagger-09-tag=%23group%2Bname%2Bfirst&tagger-10-header=assoc_actor_1&tagger-10-tag=%23group%2Bname%2Bfirst%2Bassoc&tagger-12-header=actor2&tagger-12-tag=%23group%2Bname%2Bsecond&tagger-13-header=assoc_actor_2&tagger-13-tag=%23group%2Bname%2Bsecond%2Bassoc&tagger-16-header=region&tagger-16-tag=%23region%2Bname&tagger-17-header=country&tagger-17-tag=%23country%2Bname&tagger-18-header=admin1&tagger-18-tag=%23adm1%2Bname&tagger-19-header=admin2&tagger-19-tag=%23adm2%2Bname&tagger-20-header=admin3&tagger-20-tag=%23adm3%2Bname&tagger-21-header=location&tagger-21-tag=%23loc%2Bname&tagger-22-header=latitude&tagger-22-tag=%23geo%2Blat&tagger-23-header=longitude&tagger-23-tag=%23geo%2Blon&tagger-25-header=source&tagger-25-tag=%23meta%2Bsource&tagger-27-header=notes&tagger-27-tag=%23description&tagger-28-header=fatalities&tagger-28-tag=%23affected%2Bkilled&header-row=1')
conflict_loc = conflict_loc.iloc[1:]
conflict_loc.to_csv("data/acled_unprocessed_conflict_locations.csv")'''


In [525]:
from shapely.geometry import Point
conflict_loc = pd.read_csv('data/acled_unprocessed_conflict_locations.csv')
conflict_loc_geo = gpd.GeoDataFrame(conflict_loc)
points = [Point(float(j), float(i)) for i, j in zip(conflict_loc.latitude, conflict_loc.longitude)]
geometries_acled = gpd.GeoSeries(points).buffer(0.1) # buffer is somewhat arbitrary...
# align geometries - can't know what geometry acled was using...
acled_locations = gpd.GeoDataFrame()
acled_locations['geometry'] = geometries_acled
acled_locations.crs = iq_locations.crs
acled_locations.to_crs(iq_locations.crs)
acled_locations['date'] = conflict_loc['event_date']
acled_locations.to_file('data/acled_conflict_geometries.shp')

In [ ]:
acled_temp = gpd.sjoin(iq_locations, acled_locations, 'left')
acled_temp.to_file('data/acled_joined_locations.shp')

# for now randomly assign conflict

In [527]:
#acled_temp.sample(1000).to_file('data/acled_joined_locations_sample.shp')
#pop_list  = location_csv['pop_density'] * 10 # some pop
#location_csv['pop/cap'] = [round(min(i, 0)) for i in pop_list]
pops_list = [i * 10 for i in location_csv.pop_density]
print(max(location_csv.pop_density))
location_csv['pop/cap'] = pops_list
#location_csv_new['pop/cap'] = round(location_csv_new['pop/cap'])
#location_csv = location_csv.iloc[:,:-1]
import numpy as np
conflict_list = []
conflict_type = []
for i in range(len(location_csv)):
    one = np.random.randint(0,2)
    if one > 0:
        conflict_type.append('conflict')
        one = np.random.randint(1,50)
    else:
        if np.random.randint(0,2) > 0:
            conflict_type.append('camp')
        else:
            conflict_type.append('town')
    conflict_list.append(one)
location_csv['conflict_date'] = conflict_list
location_csv['location_type'] = conflict_type
location_csv_new = location_csv.iloc[:,:-1]
location_csv_new.to_csv('flee-release/testing_locations_data.csv', index = False, header=False)


499


In [ ]:
#locations_sample = gpd.read_file('data/acled_joined_locations_sample.shp')
#conflict_locations = gpd.read_file('data/acled_joined_locations.shp')
#conflict_locations.head()
#location_csv_new['pop/cap'] = [int(i) for i in location_csv_new['pop/cap']]

In [ ]:
locations_sample = acled_temp.copy()
just_date = []
for i in locations_sample.itertuples():
    try:
        if '-' in i.date:
            just_date.append(int("".join([i for i in i.date if i != '-'])))
        else:
            just_date.append(-1)
    except:
        just_date.append(-1)
            
locations_sample['justdate'] = just_date
locations_sample.sort_values('justdate', inplace = True)
date_rank = []
datessofar = set()
current = -1
for i in locations_sample.itertuples():
    if i.justdate != -1:
        if i.justdate not in datessofar:
            current += 1
            datessofar.add(i.justdate)
        date_rank.append(current)
    else:
        date_rank.append(None)
locations_sample['conflict_start'] = date_rank
    
        
    

In [ ]:
conflict_list = []
conflicts = set()
for i in locations_sample.itertuples():
    if i.justdate >= 0:
        if str(i.Longitude) + str(i.Latitude) not in conflicts:
            conflicts.add(str(i.Longitude) + str(i.Latitude))
            conflict_list.append(i)
conflict_df = gpd.GeoDataFrame(conflict_list)
conflict_df

In [ ]:
'''location_csv_tmp = location_csv.copy()
points = [Point(i.lon, i.lat) for i in location_csv_tmp.itertuples()]
location_csv_tmp['geometry'] = points
conflict_list = []
type_list = []
for i in location_csv_tmp.itertuples():
    for j in conflict_df.itertuples():
        if i.geometry == j.geometry:
            conflict_list.append([i.geometry, j.conflict_start])
            break
    '''

#location_csv_tmp_joined = gpd.sjoin(location_csv_tmp, conflict_df, 'left')
#location_csv_tmp_joined

In [ ]:
df = pd.read_csv('flee-release/simulation_results.csv').T.reset_index()
df.columns = ['location'] +  list(df.columns[1:])
iq_locations = gpd.read_file('data/settled_locations/irq_pplp_ocha_20140722.shp')
res_df = iq_locations.merge(df, left_on = 'PNameEn', right_on = 'location')
for i in range(10):
    res_df.plot(i)